In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.impute import MissingIndicator, SimpleImputer, KNNImputer 
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_roc_curve



In [2]:
features_df = pd.read_csv('../Data/training_set_features.csv')
labels_df = pd.read_csv('../Data/training_set_labels.csv')

# check that the rows between the features and the labels match up
np.testing.assert_array_equal(features_df.index.values, labels_df.index.values)

# merge features_df and labels_df 
df = labels_df.merge(features_df, how = 'inner', on='respondent_id')

# drop duplicate 
df.drop_duplicates(inplace=True)

In [3]:
df.shape

(26707, 38)

In [4]:
df.info()

# 17  health_insurance, 36  employment_industry, 37  employment_occupation  about 50% missing 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   respondent_id                26707 non-null  int64  
 1   h1n1_vaccine                 26707 non-null  int64  
 2   seasonal_vaccine             26707 non-null  int64  
 3   h1n1_concern                 26615 non-null  float64
 4   h1n1_knowledge               26591 non-null  float64
 5   behavioral_antiviral_meds    26636 non-null  float64
 6   behavioral_avoidance         26499 non-null  float64
 7   behavioral_face_mask         26688 non-null  float64
 8   behavioral_wash_hands        26665 non-null  float64
 9   behavioral_large_gatherings  26620 non-null  float64
 10  behavioral_outside_home      26625 non-null  float64
 11  behavioral_touch_face        26579 non-null  float64
 12  doctor_recc_h1n1             24547 non-null  float64
 13  doctor_recc_seas

In [5]:
# Check balance of target variables 

print("H1N1 Vaccination (counts)")
print(df["h1n1_vaccine"].value_counts())
print()
print("Seasonal Flu Vaccination (counts)")
print(df["seasonal_vaccine"].value_counts())
print()
print("H1N1 Vaccination (percentages)")
print(df["h1n1_vaccine"].value_counts(normalize=True))
print()
print("Seasonal Flu Vaccination (percentages)")
print(df["seasonal_vaccine"].value_counts(normalize=True))

# Only 21% of people are vaccinated. H1N1 vaccination data is highly imbalanced. 
# For model tuning, use SMOTE or other methods.  



H1N1 Vaccination (counts)
0    21033
1     5674
Name: h1n1_vaccine, dtype: int64

Seasonal Flu Vaccination (counts)
0    14272
1    12435
Name: seasonal_vaccine, dtype: int64

H1N1 Vaccination (percentages)
0    0.787546
1    0.212454
Name: h1n1_vaccine, dtype: float64

Seasonal Flu Vaccination (percentages)
0    0.534392
1    0.465608
Name: seasonal_vaccine, dtype: float64


### Missing Values

In [6]:
# drop respondent_id
# df.drop('respondent_id', axis=1, inplace=True)

# report missing values 
df.isna().sum()

respondent_id                      0
h1n1_vaccine                       0
seasonal_vaccine                   0
h1n1_concern                      92
h1n1_knowledge                   116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_h1n1                2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_h1n1_vacc_effective      391
opinion_h1n1_risk                388
opinion_h1n1_sick_from_vacc      395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
s

#### Data imputation

In [7]:
# drop variables of random characters 

df.drop(['employment_industry',  'employment_occupation', 'hhs_geo_region'], axis=1, inplace=True)



In [8]:
# for socioeconomic categoircal variables, drop missing rows 

df.dropna(subset=['health_worker', 'education','income_poverty', 'marital_status', 
                    'rent_or_own', 'employment_status', 'household_adults', 
                    'household_children' ], inplace=True)


In [9]:
#features_df = pd.read_csv('../Data/training_set_features.csv')
#x_df = features_df

survey_col = ['opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc',
 'opinion_seas_vacc_effective', 'opinion_seas_risk','opinion_seas_sick_from_vacc']

behavior_col = ['behavioral_antiviral_meds', 'behavioral_face_mask',
                'behavioral_large_gatherings','behavioral_outside_home']

behavior_col_2 = ['behavioral_avoidance', 
                'behavioral_wash_hands','behavioral_touch_face']


doc_rec = ['doctor_recc_h1n1','doctor_recc_seasonal']

def impute_missing_data(dataframe, column_list, fillvalue):
    for column in column_list:
        dataframe[column].fillna(fillvalue, inplace = True)
      
    
impute_missing_data(df, survey_col, 3)
impute_missing_data(df, ['h1n1_concern'], 2)
impute_missing_data(df, ['h1n1_knowledge'], 0)
impute_missing_data(df, behavior_col, 0)
impute_missing_data(df, behavior_col_2, 1)
impute_missing_data(df, doc_rec, 0)
impute_missing_data(df, ['chronic_med_condition'], 0)
impute_missing_data(df, ['child_under_6_months'], 0)


In [10]:
df.isnull().sum()

respondent_id                     0
h1n1_vaccine                      0
seasonal_vaccine                  0
h1n1_concern                      0
h1n1_knowledge                    0
behavioral_antiviral_meds         0
behavioral_avoidance              0
behavioral_face_mask              0
behavioral_wash_hands             0
behavioral_large_gatherings       0
behavioral_outside_home           0
behavioral_touch_face             0
doctor_recc_h1n1                  0
doctor_recc_seasonal              0
chronic_med_condition             0
child_under_6_months              0
health_worker                     0
health_insurance               9400
opinion_h1n1_vacc_effective       0
opinion_h1n1_risk                 0
opinion_h1n1_sick_from_vacc       0
opinion_seas_vacc_effective       0
opinion_seas_risk                 0
opinion_seas_sick_from_vacc       0
age_group                         0
education                         0
race                              0
sex                         

#### train_test_split

In [11]:
# Define X and y, 30-70(training)

X=df.drop(['h1n1_vaccine','seasonal_vaccine'], axis=1)
y=df[['h1n1_vaccine','seasonal_vaccine']]

# Train test split 

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

X_train #15304 rows 

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,census_msa,household_adults,household_children
26356,26356,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,12 Years,Black,Female,"<= $75,000, Above Poverty",Not Married,Rent,Not in Labor Force,"MSA, Not Principle City",0.0,1.0
1925,1925,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,Some College,White,Female,"> $75,000",Not Married,Own,Employed,Non-MSA,0.0,0.0
2668,2668,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,College Graduate,White,Male,"> $75,000",Married,Own,Not in Labor Force,"MSA, Not Principle City",1.0,0.0
5325,5325,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,< 12 Years,White,Male,"<= $75,000, Above Poverty",Not Married,Rent,Not in Labor Force,Non-MSA,0.0,0.0
841,841,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Unemployed,"MSA, Principle City",0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14593,14593,2.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,Some College,White,Male,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,Non-MSA,1.0,0.0
26345,26345,2.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,< 12 Years,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,Non-MSA,0.0,0.0
6555,6555,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,College Graduate,White,Male,"> $75,000",Married,Rent,Employed,"MSA, Principle City",1.0,0.0
1035,1035,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,Non-MSA,3.0,2.0


#### Imputing Health Insurance 

In [12]:
X_train.shape

(15304, 33)

In [13]:
# create dummies for objects to use for KNNImputer

X_train_obj = X_train.select_dtypes(include='object')

# one hot encoding for objects 
ohe = OneHotEncoder(drop='first', sparse=False)

dums = ohe.fit_transform(X_train_obj)

dums_df = pd.DataFrame(dums,
                       columns=ohe.get_feature_names(),
                       index=X_train_obj.index)

dums_df
dums_df.shape #15304 rows 



(15304, 19)

In [14]:
# Imputing helath insurance with KNNImputer 
#knnimpute = KNNImputer()
#X_train_imputed = pd.DataFrame(knnimpute.fit_transform(X_train), columns = X_train.columns)

#X_train_imputed

#X_train_imputed.health_insurance.value_counts()
#X_train_imputed.health_insurance = (X_train_imputed.health_insurance > 0.5).astype(int)
#X_train_imputed.health_insurance.value_counts()

In [15]:
dums_df

,x0_35 - 44 Years,x0_45 - 54 Years,x0_55 - 64 Years,x0_65+ Years,x1_< 12 Years,x1_College Graduate,x1_Some College,x2_Hispanic,x2_Other or Multiple,x2_White,x3_Male,"x4_> $75,000",x4_Below Poverty,x5_Not Married,x6_Rent,x7_Not in Labor Force,x7_Unemployed,"x8_MSA, Principle City",x8_Non-MSA
26356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
1925,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2668,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5325,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
841,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14593,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
26345,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
6555,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1035,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [16]:
# Imputation using other socio-econmic features  

dums_df_insurance = pd.concat([X_train.health_insurance, dums_df], axis=1)

knnimpute = KNNImputer()
dums_df_insurance_imputed = pd.DataFrame(knnimpute.fit_transform(dums_df_insurance), 
                                         columns = dums_df_insurance.columns,
                                        index=dums_df_insurance.index)

dums_df_insurance_imputed # 15304 rows


,health_insurance,x0_35 - 44 Years,x0_45 - 54 Years,x0_55 - 64 Years,x0_65+ Years,x1_< 12 Years,x1_College Graduate,x1_Some College,x2_Hispanic,x2_Other or Multiple,x2_White,x3_Male,"x4_> $75,000",x4_Below Poverty,x5_Not Married,x6_Rent,x7_Not in Labor Force,x7_Unemployed,"x8_MSA, Principle City",x8_Non-MSA
26356,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
1925,0.8,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2668,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5325,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
841,0.8,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14593,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
26345,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
6555,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1035,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [17]:
dums_df_insurance_imputed.health_insurance.value_counts()

1.0    12105
0.8     1278
0.0     1046
0.6      582
0.4      216
0.2       77
Name: health_insurance, dtype: int64

In [18]:
# convert to 0,1 value 


dums_df_insurance_imputed.health_insurance = (dums_df_insurance_imputed.health_insurance
                                              > 0.5).astype(int)

dums_df_insurance_imputed.health_insurance.value_counts()


1    13965
0     1339
Name: health_insurance, dtype: int64

In [19]:
dums_df_insurance_imputed


,health_insurance,x0_35 - 44 Years,x0_45 - 54 Years,x0_55 - 64 Years,x0_65+ Years,x1_< 12 Years,x1_College Graduate,x1_Some College,x2_Hispanic,x2_Other or Multiple,x2_White,x3_Male,"x4_> $75,000",x4_Below Poverty,x5_Not Married,x6_Rent,x7_Not in Labor Force,x7_Unemployed,"x8_MSA, Principle City",x8_Non-MSA
26356,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
1925,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2668,1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5325,1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
841,1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14593,1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
26345,1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
6555,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1035,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [20]:
X_train_obj.columns

Index(['age_group', 'education', 'race', 'sex', 'income_poverty',
       'marital_status', 'rent_or_own', 'employment_status', 'census_msa'],
      dtype='object')

In [21]:
# Drop the old object columns and health insurance column from X_train 
# and concat the imputed df. 

X_train_1 = X_train.drop(X_train_obj.columns, axis=1)
X_train_1 = X_train_1.drop('health_insurance', axis=1)

X_train_imputed = pd.concat([X_train_1, dums_df_insurance_imputed], axis=1)



X_train_imputed


,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,x2_White,x3_Male,"x4_> $75,000",x4_Below Poverty,x5_Not Married,x6_Rent,x7_Not in Labor Force,x7_Unemployed,"x8_MSA, Principle City",x8_Non-MSA
26356,26356,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
1925,1925,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2668,2668,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5325,5325,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
841,841,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14593,14593,2.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
26345,26345,2.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
6555,6555,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1035,1035,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [22]:
X_train_imputed.columns

Index(['respondent_id', 'h1n1_concern', 'h1n1_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'household_adults',
       'household_children', 'health_insurance', 'x0_35 - 44 Years',
       'x0_45 - 54 Years', 'x0_55 - 64 Years', 'x0_65+ Years', 'x1_< 12 Years',
       'x1_College Graduate', 'x1_Some College', 'x2_Hispanic',
       'x2_Other or Multiple', 'x2_White', 'x3_Male', 'x4_> $75,000',
       'x4_Below Poverty', 'x5_Not Married', 'x6_Rent',
       'x7_Not in Labor Force', 'x7_Unemployed', 'x8_MSA, Principl

In [23]:
X_train_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15304 entries, 26356 to 19277
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   respondent_id                15304 non-null  int64  
 1   h1n1_concern                 15304 non-null  float64
 2   h1n1_knowledge               15304 non-null  float64
 3   behavioral_antiviral_meds    15304 non-null  float64
 4   behavioral_avoidance         15304 non-null  float64
 5   behavioral_face_mask         15304 non-null  float64
 6   behavioral_wash_hands        15304 non-null  float64
 7   behavioral_large_gatherings  15304 non-null  float64
 8   behavioral_outside_home      15304 non-null  float64
 9   behavioral_touch_face        15304 non-null  float64
 10  doctor_recc_h1n1             15304 non-null  float64
 11  doctor_recc_seasonal         15304 non-null  float64
 12  chronic_med_condition        15304 non-null  float64
 13  child_under_

In [24]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15304 entries, 26356 to 19277
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   h1n1_vaccine      15304 non-null  int64
 1   seasonal_vaccine  15304 non-null  int64
dtypes: int64(2)
memory usage: 358.7 KB


In [25]:
#Imputing Health insurance for the test set

X_test_obj = X_test.select_dtypes(include='object')

# one hot encoding for objects 
# ohe = OneHotEncoder(drop='first', sparse=False)

dumstest = ohe.transform(X_test_obj)

dums_df2 = pd.DataFrame(dumstest,
                       columns=ohe.get_feature_names(),
                       index=X_test_obj.index)

dums_df2
dums_df2.shape #15304 rows 



(6559, 19)

In [26]:
dums_df_insurancetest = pd.concat([X_test.health_insurance, dums_df2], axis=1)

knnimpute = KNNImputer()
dums_df_insurance_imputedtest = pd.DataFrame(knnimpute.fit_transform(dums_df_insurancetest), 
                                         columns = dums_df_insurancetest.columns,
                                        index=dums_df_insurancetest.index)

dums_df_insurance_imputedtest

,health_insurance,x0_35 - 44 Years,x0_45 - 54 Years,x0_55 - 64 Years,x0_65+ Years,x1_< 12 Years,x1_College Graduate,x1_Some College,x2_Hispanic,x2_Other or Multiple,x2_White,x3_Male,"x4_> $75,000",x4_Below Poverty,x5_Not Married,x6_Rent,x7_Not in Labor Force,x7_Unemployed,"x8_MSA, Principle City",x8_Non-MSA
15045,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
14123,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
25010,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
24771,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
23611,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7735,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
20174,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19888,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
22054,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [27]:

dums_df_insurance_imputedtest.health_insurance = (dums_df_insurance_imputedtest.health_insurance
                                              > 0.5).astype(int)

dums_df_insurance_imputedtest.health_insurance.value_counts()


1    5989
0     570
Name: health_insurance, dtype: int64

In [28]:
X_test_1 = X_test.drop(X_test_obj.columns, axis=1)
X_test_1 = X_test_1.drop('health_insurance', axis=1)

X_test_imputed = pd.concat([X_test_1, dums_df_insurance_imputedtest], axis=1)



X_test_imputed

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,x2_White,x3_Male,"x4_> $75,000",x4_Below Poverty,x5_Not Married,x6_Rent,x7_Not in Labor Force,x7_Unemployed,"x8_MSA, Principle City",x8_Non-MSA
15045,15045,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
14123,14123,3.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
25010,25010,3.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
24771,24771,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
23611,23611,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7735,7735,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
20174,20174,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19888,19888,2.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
22054,22054,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [30]:
#drop respondentID for training and test so it doesn't influence features and model

X_train2 = X_train_imputed.drop('respondent_id', axis=1)
X_test2 = X_test_imputed.drop('respondent_id', axis=1)
# y_train = y_train.drop('respondent_id', axis=1)
# y_test = y_test.drop('respondent_id', axis=1)

## Now that the training data is clean and Health Insurance has been imputed, we can begin to create our training models


#### Model Creation

In [31]:
#KNN
from sklearn.neighbors import KNeighborsClassifier


knn = KNeighborsClassifier()
knn.fit(X_train2, y_train)

KNeighborsClassifier()

In [32]:
from sklearn.metrics import accuracy_score

y_pred = knn.predict(X_test2)
accuracy_score(y_test, y_pred)

0.6135081567312091

In [33]:
#model turning for KNN

def find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=25):
    best_k = 0
    best_score = 0.0
    for k in range(min_k, max_k+1, 2):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        f1 = f1_score(y_test, y_pred, average='weighted')
        if f1 > best_score:
            best_k = k
            best_score = f1
    
    print("Best Value for k: {}".format(best_k))
    print("F1-Score: {}".format(best_score))

In [34]:
from sklearn.metrics import f1_score

find_best_k(X_train2, y_train, X_test2, y_test, min_k=1, max_k=49)

Best Value for k: 21
F1-Score: 0.6384036290315418


In [35]:
#best k=1, can we change metrics to find best performer?
def find_best_metric(X_train, y_train, X_test, y_test, metric_list):
    best_metric = ''
    best_score = 0
    for val in metric_list:
        knn2 = KNeighborsClassifier(n_neighbors=21, metric=val)
        knn2.fit(X_train, y_train)
        y_pred = knn2.predict(X_test)
        f1 = f1_score(y_test, y_pred, average='weighted')
        if f1 > best_score:
            best_metric = val
            best_score = f1
    print("Best metric: {}".format(best_metric))  
    print("F1-Score: {}".format(best_score))

In [37]:
metrics = ['euclidean', 'manhattan', 'minkowski']

find_best_metric(X_train2, y_train, X_test2, y_test, metrics)

Best metric: manhattan
F1-Score: 0.5856439464540965


In [38]:
#kNN doesn't seem to be a good model. attempting decisiontreeclassifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

dt = DecisionTreeClassifier(criterion='entropy')

dt.fit(X_train2, y_train)


DecisionTreeClassifier(criterion='entropy')

In [44]:
# fig, axes = plt.subplots(nrows = 1,ncols = 1, figsize = (3,3), dpi=300)
# plot_tree(dt,
#                feature_names = X_train_imputed.columns, 
#                class_names=np.unique(y).astype('str'),
#                filled = True)
# plt.show()

KeyboardInterrupt: 

In [39]:
y_dt_pred = dt.predict(X_test2)
accuracy_score(y_test, y_dt_pred)

0.5545052599481628

#### Insight: after model tuning with knn, we do not seem to have a good score. This could be due to the nature of some features not being binary. We will one hot encode the survey items and reattempt knn or other models.

In [41]:
#multioutputclassifier attempt

from sklearn.multioutput import MultiOutputClassifier

moc = MultiOutputClassifier(estimator=KNeighborsClassifier()).fit(X_train2, y_train)

In [42]:
y_pred = moc.predict_proba(X_test2)